In [4]:
import pandas as pd
import numpy as np
sp = pd.read_csv(r"C:\Users\VANDHANA\Desktop\project\spotify_dataset.csv")
top = pd.read_excel(r"C:\Users\VANDHANA\Desktop\project\top10.xlsx")
top.dropna(inplace = True)
sp.dropna(inplace = True)
spc = sp[['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness','Liveness', 'Valence']]
spc = spc.apply(pd.to_numeric,errors = 'coerce')
spc = spc[['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness','Liveness', 'Valence']].astype(float)
topc = top[[ 'dnce', 'nrgy', 'dB','spch', 'acous', 'live', 'val']]
sp.drop(['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness','Liveness', 'Valence'],axis = 1,inplace = True)
top.drop([ 'dnce', 'nrgy', 'dB','spch', 'acous', 'live', 'val'],axis = 1,inplace = True)
for i in topc[[ 'dnce', 'nrgy','spch', 'acous', 'live', 'val']]:
    topc[i] = topc[i] / topc[i].abs().max()
sp['Song Name'] = sp['Song Name'].str.lower()
top['title'] = top['title'].str.lower()
sp['Artist'] = sp['Artist'].str.lower()
top['artist'] = top['artist'].str.lower()
top.fillna('0',inplace = True)
print(topc.shape,spc.shape)

(603, 7) (1556, 7)


C:\Users\VANDHANA\AppData\Local\Temp/ipykernel_9000/1200821941.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topc[i] = topc[i] / topc[i].abs().max()


In [8]:
print("This song recommender works for all popular english songs from 2010 to 2019")
song = str(input("Enter a song name you listen:"))
artist = str(input("Your favourite artist:"))
genre = str(input("Genre you like:"))
genre = genre.lower()
artist = artist.lower()
song = song.lower()
s = sp[sp['Artist'] == artist].sort_values(by = ['Streams'],ascending = False)
if(len(s.index) < 3):
    spp = s.head(len(s.index))
else:
    spp = s.head(3)
s = sp[sp['Genre'].str.contains(genre)].sort_values(by = ['Streams'],ascending = False)
if(len(s.index) < 3):
    spp = spp.append(s.head(len(s.index)))
else:
    spp = spp.append(s.head(3))   
s =  top[top['artist'] == artist]
if(len(s.index) < 3):
    topp = s.sample(len(s.index))
else:
    topp = s.sample(3)  
s = top[top['top genre'].str.contains(genre)]
if(len(s.index) < 3):
    topp = topp.append(s.sample(len(s.index)))
else:
     topp = topp.append(s.sample(3))           
spp = spp.sort_values(by=['Streams'],ascending = False)
e = 0
if((len(spp.index) == 0) or (len(topp.index) == 0)):
    print("Give genre and artist as per the constraints!")
    e = 1

This song recommender works for all popular english songs from 2010 to 2019
Enter a song name you listen:gorgeous
Your favourite artist:zyan
Genre you like:hip hop


In [9]:
a = sp[sp['Song Name'] == (song)].index
b = top[top['title'] == (song)].index
spcp = []
topcp = []
if(len(a)):
    spcp = list(spc.loc[a[0]])
elif(e == 0): 
    spcp = spc.loc[spp.index]
    spcp = list(spcp.mean())
if(len(b)):
    topcp = list(topc.loc[b[0]])
elif(e == 0):    
    topcp = topc.loc[topp.index]
    topcp = list(topcp.mean())    
if((len(spcp) == 0 or len(topcp) == 0) and (e == 1)):
    print("Provide valid song name also!")
    exit()
print(spcp,topcp)    

[0.6206666666666666, 0.6919999999999998, -4.778333333333333, 0.25966666666666666, 0.2977666666666667, 0.26966666666666667, 0.6823333333333333] [0.7079037800687286, 0.7040816326530611, -6.666666666666667, 0.38888888888888884, 0.14814814814814817, 0.1981981981981982, 0.36054421768707484]


In [10]:
import random
def mink_dis(a,b,p = 1):
    dim = len(a)
    dis = 0
    for d in range(dim):
        dis += abs(a[d] - b[d])**p
    dis = dis**(1/p)
    return dis
dist = []
for i in spc.index:
    dist.append(mink_dis(spcp,spc.iloc[i]))
spc_dists = pd.DataFrame(data=dist,index=spc.index,columns=['dist'])
spc_dists = spc_dists.sort_values(by=['dist'],axis = 0)[2:4]
dist = []
for i in topc.index:
    dist.append(mink_dis(topcp,topc.iloc[i]))
topc_dists = pd.DataFrame(data=dist,index=topc.index,columns=['dist'])
topc_dists = topc_dists.sort_values(by=['dist'],axis = 0)[2:4]
sp_final = sp.loc[spc_dists.index]
top_final = top.loc[topc_dists.index]
print("songs you may like:")
l = []
if(len(spp.index) < 3):
    l.append(spp['Song Name'].head(len(spp.index)))
else:
    l.append(spp['Song Name'].head(3))
if(len(topp.index ) < 3):    
    l.append(random.sample(sorted(topp['title']),len(topp.index)))
else:
    l.append(random.sample(sorted(topp['title']),3))
l.append(list(sp_final['Song Name'].values))
l.append(list(top_final['title'].values))
for i in l:
    for j in i:
        print(j)

songs you may like:
time flies
hot wind blows (feat. lil wayne)
nicky jam: bzrp music sessions, vol. 41
castle walls (feat. christina aguilera)
lemon
see you again (feat. charlie puth)
11:00 pm
don don
say something
make me... (feat. g-eazy)
